In [1]:
import geopandas as gpd
import networkx as nx
import fiona
import tqdm
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

from shapely.ops import cascaded_union
from shapely.wkt import loads 
from shapely.geometry import Point
from scipy.spatial import cKDTree

# 使用データ

In [ ]:
"""
'Lakshminiya'データないエラー
"""

In [5]:
district = gpd.read_file(r'C:\Users\kurokiso\Desktop\道路ネットワーク\取得情報\hermes_NPL_new_wgs\hermes_NPL_new_wgs_3.shp')
use_dis_list = ['Bhadrapur', 'Dharan', 'Biratnagar', 'Rajbiraj', 'Lahan', 'Janakpur', 'Lakshminiya', 'Jaleswor', 'Gaushala', 'Malangawa', 'Gaur', 'Hetauda', 'Kalaiya', 'Birgunj', 'Bharatpur', 'Pokhara Lekhnath', 'Butwal', 'Siddharthanagar', 'Kapilbastu', 'Krishnanagar', 'Nepalgunj', 'Dhangadhi', 'Bhimdatta', ]
water = gpd.read_file(r'C:\Users\kurokiso\Desktop\道路ネットワーク\water_gpkg\all.gpkg')
source = gpd.read_file(r'C:\Users\kurokiso\Desktop\道路ネットワーク\water_gpkg\sou_E.gpkg')
road = gpd.read_file(r'C:\Users\kurokiso\Desktop\道路ネットワーク\npl-rdsl-trans-25k-50k-sdn-wgs84\npl_rdsl_trans_25K_50K_sdn_wgs84.shp')
road = road.reset_index()

In [7]:
def ckdnearest(gdA, gdB):

    nA = np.array(list(gdA.geometry.apply(lambda x: (x.x, x.y))))
    nB = np.array(list(gdB.geometry.apply(lambda x: (x.x, x.y))))
    btree = cKDTree(nB)
    dist, idx = btree.query(nA, k=1)
    gdB_nearest = gdB.iloc[idx].drop(columns="geometry").reset_index(drop=True)
    gdf = pd.concat(
        [
            gdA.reset_index(drop=True),
            gdB_nearest,
            pd.Series(dist, name='dist')
        ], 
        axis=1)

    return gdf

In [190]:
use_dis_list = ['Bharatpur', 'Pokhara Lekhnath', 'Butwal', 'Siddharthanagar', 'Kapilbastu', 'Krishnanagar', 'Nepalgunj', 'Dhangadhi', 'Bhimdatta', ]

In [191]:
for i in use_dis_list:
    local = i
    os.mkdir(local)
    # 対象地区の取り出し
    df_dis = district[district['LOCAL'] == i]
    water_clipped = gpd.clip(water, df_dis)
    source_clipped = gpd.clip(source, df_dis)
    road_clipped = gpd.clip(road, df_dis)
    gpd.GeoDataFrame.to_file(water_clipped, "{}/water.gpkg".format(local), driver="GPKG", encoding='utf-8')
    gpd.GeoDataFrame.to_file(source_clipped, "{}/source.gpkg".format(local), driver="GPKG", encoding='utf-8')
    gpd.GeoDataFrame.to_file(road_clipped[['index','LENGTH', 'geometry']], "{}/road.gpkg".format(local), driver="GPKG", encoding='utf-8')    
    
    # 始点と終点の抜き出し
    id_list = []
    index_list = []
    geopoint_list = []
    start_id = 0
    with fiona.open('{}/road.gpkg'.format(local)) as lines:
        for line in lines:
            geopoint_list.append(str(Point(line['geometry']['coordinates'][0])))
            id_list.append(start_id)
            start_id += 1
            index_list.append(line['properties']['index'])
            geopoint_list.append(str(Point(line['geometry']['coordinates'][-1])))
            id_list.append(start_id)
            start_id += 1
            index_list.append(line['properties']['index'])
    df = pd.DataFrame({'id': id_list, 'index': index_list, 'geometry': geopoint_list})
    df = gpd.GeoDataFrame(df, geometry=[loads(wkt) for wkt in df['geometry']])
    df.to_file('{}/vertices.gpkg'.format(local), driver='GPKG')
    
    # 重なっている点を取り出す
    df['X'] = df['geometry'].x
    df['Y'] = df['geometry'].y
    df2 = pd.DataFrame(df.groupby(['X', 'Y']).agg(id=('id', lambda x: list(x.value_counts().index))))
    dup_list = df2['id'].values
    
    # uniqeu点のみ取り出す
    unique_list = []
    for i in dup_list:
        unique_list.append(i[0])
    unique_df = df.set_index('id').iloc[unique_list, :].reset_index()[['id', 'index', 'geometry']]
    unique_df.to_file('{}/uqnique_vertices.gpkg'.format(local), driver='GPKG')

    # 2点と距離の情報
    start_id_list = []
    end_id_list = []
    length_list = []
    start_id = 0
    with fiona.open('{}/road.gpkg'.format(local)) as lines:
        for line in lines:
            start_id_list.append(start_id)
            start_id += 1
            end_id_list.append(start_id)
            start_id += 1
            length_list.append(line['properties']['LENGTH'])    
    distance = pd.DataFrame({'start_id': start_id_list, 'end_id': end_id_list, 'length': length_list})
    
    # uniqueの点に統一する
    dic = {}
    for i in dup_list:
        for v in i:
            dic[v] = i[0]
    distance['start_id'] = distance['start_id'].apply(lambda x: dic[x])
    distance['end_id'] = distance['end_id'].apply(lambda x: dic[x])
    
    # 隣接行列作成
    node_num = len(id_list)
    INF = 10**5 - 1
    W = [[INF] * node_num for _ in range(node_num)]
    for i in range(distance.shape[0]):
        W[distance.loc[i, 'start_id']][distance.loc[i, 'end_id']] = distance.loc[1, 'length']
        W[distance.loc[i, 'end_id']][distance.loc[i, 'start_id']] = distance.loc[1, 'length']
    
    # 井戸最近棒
    source_nearest = ckdnearest(source_clipped, unique_df)[['Code', 'id']]
    
    # 水道の最近棒
    water_nearest = ckdnearest(water_clipped, unique_df)[['Rowid', 'id']]
    
    # 経路まとめ
    node_num = len(id_list)
    INF = 0
    W = [[INF] * node_num for _ in range(node_num)]
    for i in range(distance.shape[0]):
        W[distance.loc[i, 'start_id']][distance.loc[i, 'end_id']] = distance.loc[i, 'length']
        W[distance.loc[i, 'end_id']][distance.loc[i, 'start_id']] = distance.loc[i, 'length']

    data = np.array(W)
    G=nx.from_numpy_matrix(data)

    water_rowid_list = []
    nearest_dist_list = []
    nearest_unite_list = []
    for i in range(water_nearest.shape[0]):
        water_rowid_list.append(water_nearest.loc[i, 'Rowid'])
        water_id = water_nearest.loc[i, 'id']
        dist_list = []
        unite_list = []
        for v in range(source_nearest.shape[0]):
            source_id = source_nearest.loc[v, 'id']
            nearest_point = nx.shortest_path(G, source=water_id, target=source_id, weight='weight')
            nearest_dis = nx.shortest_path_length(G, source=water_id, target=source_id, weight='weight')
            dist_list.append(nearest_dis)
            unite_list.append(nearest_point)
        min_dist_index = dist_list.index(min(dist_list))
        min_dist = dist_list[min_dist_index]
        min_unite = unite_list[min_dist_index]
        nearest_dist_list.append(min_dist)
        nearest_unite_list.append(min_unite)
    output = pd.DataFrame({'Rowid': water_rowid_list, 'nearest_source_dist': nearest_dist_list, 'nearest_unite_list': nearest_unite_list})
    output.to_csv('{}/output.csv'.format(local))
    
    # おまけ unite 点の出力
    for i in range(output.shape[0]):
        point_list = output.loc[i, 'nearest_unite_list']
        rowid = output.loc[i, 'Rowid']
        nearest_point_info = unique_df.set_index('id').loc[point_list, :].reset_index()[['id', 'index', 'geometry']]
        nearest_point_info.to_file('{}/unite_point{}.gpkg'.format(local, rowid), driver='GPKG')

C:\Users\kurokiso\anaconda3\lib\site-packages\geopandas\geoseries.py:358: UserWarning: GeoSeries.notna() previously returned False for both missing (None) and empty geometries. Now, it only returns False for missing values. Since the calling GeoSeries contains empty geometries, the result has changed compared to previous versions of GeoPandas.
Given a GeoSeries 's', you can use '~s.is_empty & s.notna()' to get back the old behaviour.

To further ignore this warning, you can do: 
import warnings; warnings.filterwarnings('ignore', 'GeoSeries.notna', UserWarning)
  return self.notna()
C:\Users\kurokiso\anaconda3\lib\site-packages\geopandas\geoseries.py:358: UserWarning: GeoSeries.notna() previously returned False for both missing (None) and empty geometries. Now, it only returns False for missing values. Since the calling GeoSeries contains empty geometries, the result has changed compared to previous versions of GeoPandas.
Given a GeoSeries 's', you can use '~s.is_empty & s.notna()' to g

# カトマンズ

In [2]:
df_dis_list = ['Tarakeshwor', 'Tokha', 'Budhanilakantha', 'Gokarneshwor', 'Kageshwori Manahora', 'Changunarayan', 'Madhyapur Thimi', 'Bhaktapur', 'Kathmandu', 'Kirtipur', 'Lalitpur', 'Dakshinkali', 'Godawari']
df_dis_list = ['Birgunj']

In [6]:
df_dis = district[district['LOCAL'].isin(df_dis_list)]
df_dis = gpd.GeoSeries(cascaded_union(df_dis.geometry))

local = 'kathmandu'
local = 'Birgunj'
os.mkdir(local)
water_clipped = gpd.clip(water, df_dis)
source_clipped = gpd.clip(source, df_dis)
road_clipped = gpd.clip(road, df_dis)
gpd.GeoDataFrame.to_file(water_clipped, "{}/water.gpkg".format(local), driver="GPKG", encoding='utf-8')
gpd.GeoDataFrame.to_file(source_clipped, "{}/source.gpkg".format(local), driver="GPKG", encoding='utf-8')
gpd.GeoDataFrame.to_file(road_clipped[['index','LENGTH', 'geometry']], "{}/road.gpkg".format(local), driver="GPKG", encoding='utf-8')    

C:\Users\kurokiso\anaconda3\lib\site-packages\geopandas\geoseries.py:358: UserWarning: GeoSeries.notna() previously returned False for both missing (None) and empty geometries. Now, it only returns False for missing values. Since the calling GeoSeries contains empty geometries, the result has changed compared to previous versions of GeoPandas.
Given a GeoSeries 's', you can use '~s.is_empty & s.notna()' to get back the old behaviour.

To further ignore this warning, you can do: 
import warnings; warnings.filterwarnings('ignore', 'GeoSeries.notna', UserWarning)
  return self.notna()


In [8]:
# 始点と終点の抜き出し
local = 'Birgunj'
id_list = []
index_list = []
geopoint_list = []
start_id = 0
with fiona.open('{}/road.gpkg'.format(local)) as lines:
    for line in lines:
        geopoint_list.append(str(Point(line['geometry']['coordinates'][0])))
        id_list.append(start_id)
        start_id += 1
        index_list.append(line['properties']['index'])
        geopoint_list.append(str(Point(line['geometry']['coordinates'][-1])))
        id_list.append(start_id)
        start_id += 1
        index_list.append(line['properties']['index'])
df = pd.DataFrame({'id': id_list, 'index': index_list, 'geometry': geopoint_list})
df = gpd.GeoDataFrame(df, geometry=[loads(wkt) for wkt in df['geometry']])
df.to_file('{}/vertices.gpkg'.format(local), driver='GPKG')
print(df)

# 重なっている点を取り出す
df['X'] = df['geometry'].x
df['Y'] = df['geometry'].y
df2 = pd.DataFrame(df.groupby(['X', 'Y']).agg(id=('id', lambda x: list(x.value_counts().index))))
dup_list = df2['id'].values
print(df2)

# uniqeu点のみ取り出す
unique_list = []
for i in dup_list:
    unique_list.append(i[0])
unique_df = df.set_index('id').iloc[unique_list, :].reset_index()[['id', 'index', 'geometry']]
unique_df.to_file('{}/uqnique_vertices.gpkg'.format(local), driver='GPKG')
print(unique_df)

# 2点と距離の情報
start_id_list = []
end_id_list = []
length_list = []
start_id = 0
with fiona.open('{}/road.gpkg'.format(local)) as lines:
    for line in lines:
        start_id_list.append(start_id)
        start_id += 1
        end_id_list.append(start_id)
        start_id += 1
        length_list.append(line['properties']['LENGTH'])    
distance = pd.DataFrame({'start_id': start_id_list, 'end_id': end_id_list, 'length': length_list})

# uniqueの点に統一する
dic = {}
for i in dup_list:
    for v in i:
        dic[v] = i[0]
distance['start_id'] = distance['start_id'].apply(lambda x: dic[x])
distance['end_id'] = distance['end_id'].apply(lambda x: dic[x])

# 隣接行列作成
node_num = len(id_list)
INF = 10**5 - 1
W = [[INF] * node_num for _ in range(node_num)]
for i in range(distance.shape[0]):
    W[distance.loc[i, 'start_id']][distance.loc[i, 'end_id']] = distance.loc[1, 'length']
    W[distance.loc[i, 'end_id']][distance.loc[i, 'start_id']] = distance.loc[1, 'length']

# 井戸最近棒
source_nearest = ckdnearest(source_clipped, unique_df)[['Code', 'id']]

# 水道の最近棒
water_nearest = ckdnearest(water_clipped, unique_df)[['Rowid', 'id']]

# 経路まとめ
node_num = len(id_list)
INF = 0
W = [[INF] * node_num for _ in range(node_num)]
for i in range(distance.shape[0]):
    W[distance.loc[i, 'start_id']][distance.loc[i, 'end_id']] = distance.loc[i, 'length']
    W[distance.loc[i, 'end_id']][distance.loc[i, 'start_id']] = distance.loc[i, 'length']

data = np.array(W)
G=nx.from_numpy_matrix(data)

        id   index                   geometry
0        0  125406  POINT (84.94804 27.12536)
1        1  125406  POINT (84.94805 27.12543)
2        2  125410  POINT (84.90588 27.12543)
3        3  125410  POINT (84.90583 27.12506)
4        4  125411  POINT (84.95261 27.12474)
...    ...     ...                        ...
3329  3329  564034  POINT (84.86640 26.98451)
3330  3330  564039  POINT (84.86878 26.98409)
3331  3331  564039  POINT (84.86768 26.98371)
3332  3332  564041  POINT (84.87218 26.99115)
3333  3333  564041  POINT (84.87269 26.99138)

[3334 rows x 3 columns]
                                     id
X         Y                            
84.812452 27.036582              [2313]
84.813848 27.027517              [2443]
84.814086 27.036261  [2311, 2331, 2312]
84.814689 27.032124              [2442]
84.814817 27.038965              [2310]
...                                 ...
84.956471 27.133148  [3134, 2957, 2960]
84.956806 27.133004              [2961]
84.956906 27.134796    

In [9]:
road

,index,FNODE_,TNODE_,LPOLY_,RPOLY_,LENGTH,TRANS_LN_,TRANS_LN_I,FCODE,FEATURES,Shape_Leng,geometry
0,0,3.0,4.0,1.0,1.0,946.207356,1.0,1.0,10231,Local Trails,0.008864,"LINESTRING (85.86285 28.07087, 85.86407 28.069..."
1,1,4.0,6.0,2.0,1.0,1298.863346,2.0,2.0,10231,Local Trails,0.011991,"LINESTRING (85.86770 28.06376, 85.86848 28.062..."
2,2,4.0,6.0,1.0,2.0,1582.131701,3.0,3.0,10231,Local Trails,0.014991,"LINESTRING (85.86770 28.06376, 85.86827 28.063..."
3,3,8.0,7.0,3.0,1.0,278.599601,4.0,4.0,10221,Main Trail,0.002639,"LINESTRING (85.77420 28.04159, 85.77382 28.041..."
4,4,11.0,7.0,1.0,3.0,802.662274,5.0,5.0,10231,Local Trails,0.007485,"LINESTRING (85.76874 28.03739, 85.76873 28.037..."
...,...,...,...,...,...,...,...,...,...,...,...,...
643827,643827,3939.0,3936.0,0.0,0.0,45.927640,1422.0,1422.0,10621,Crossing Ford,0.000467,"LINESTRING (85.11936 27.99578, 85.11889 27.99581)"
643828,643828,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0,Road Tunnel 1,0.076115,"LINESTRING (81.40624 28.37593, 81.38948 28.383..."
643829,643829,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0,Road Tunnel 1,0.070515,"LINESTRING (81.37062 28.42990, 81.37945 28.426..."
643830,643830,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0,Road Tunnel 1,0.013548,"LINESTRING (81.40497 28.37649, 81.41755 28.37146)"


In [113]:
water_rowid_list = []
nearest_dist_list = []
nearest_unite_list = []
for i in range(water_nearest.shape[0]):
    water_id = water_nearest.loc[i, 'id']
    dist_list = []
    unite_list = []
    for v in range(source_nearest.shape[0]):
        source_id = source_nearest.loc[v, 'id']
        try:
            nearest_point = nx.shortest_path(G, source=water_id, target=source_id, weight='weight')
            nearest_dis = nx.shortest_path_length(G, source=water_id, target=source_id, weight='weight')
            dist_list.append(nearest_dis)
            unite_list.append(nearest_point)
        except:
            continue
    try:
        min_dist_index = dist_list.index(min(dist_list))
        min_dist = dist_list[min_dist_index]
        min_unite = unite_list[min_dist_index]
        nearest_dist_list.append(min_dist)
        nearest_unite_list.append(min_unite)
        water_rowid_list.append(water_nearest.loc[i, 'Rowid'])
        print('-----------------------------------------------------------------------------------------------')
        print('water', water_id)
        print(min_dist)
        print(min_unite)
    except:
        continue
output = pd.DataFrame({'Rowid': water_rowid_list, 'nearest_source_dist': nearest_dist_list, 'nearest_unite_list': nearest_unite_list})
output.to_csv('{}/output.csv'.format(local))

# おまけ unite 点の出力
for i in range(output.shape[0]):
    point_list = output.loc[i, 'nearest_unite_list']
    rowid = output.loc[i, 'Rowid']
    nearest_point_info = unique_df.set_index('id').loc[point_list, :].reset_index()[['id', 'index', 'geometry']]
    nearest_point_info.to_file('{}/unite_point{}.gpkg'.format(local, rowid), driver='GPKG')

-----------------------------------------------------------------------------------------------
water 923
79.56329
[923, 922]
-----------------------------------------------------------------------------------------------
water 1918
689.70919
[1918, 1919, 1930, 1943, 1946, 1951, 1179]
-----------------------------------------------------------------------------------------------
water 942
678.0266100000001
[942, 935, 1871, 923, 922]
